In [1]:
from models.graph import Graph, load_graph
from models.interaction_network import InteractionNetwork
from utils import *
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import time
import h5py

prep, pt_cut = 'LP', 5
graph_dir = "../IN_{0}_{1}/".format(prep, pt_cut)
model_path = './trained_models/LP_2_1500_wide_noPhi_epoch300.pt'
data_path = '../data'

graphs = get_graphs(graph_dir)
print(len(graphs))
objects, sender_relations, receiver_relations, relation_info, y = get_inputs(graphs)

weights = torch.load(model_path)

1000


In [3]:
object_dim, relation_dim, effect_dim = 3, 1, 1
interaction_network = InteractionNetwork(object_dim, relation_dim, effect_dim)

interaction_network.load_state_dict(weights)
interaction_network.eval()

InteractionNetwork(
  (relational_model): RelationalModel(
    (layers): Sequential(
      (0): Linear(in_features=7, out_features=250, bias=True)
      (1): ReLU()
      (2): Linear(in_features=250, out_features=250, bias=True)
      (3): ReLU()
      (4): Linear(in_features=250, out_features=250, bias=True)
      (5): ReLU()
      (6): Linear(in_features=250, out_features=1, bias=True)
      (7): Sigmoid()
    )
  )
  (object_model): ObjectModel(
    (layers): Sequential(
      (0): Linear(in_features=4, out_features=200, bias=True)
      (1): ReLU()
      (2): Linear(in_features=200, out_features=200, bias=True)
      (3): ReLU()
      (4): Linear(in_features=200, out_features=3, bias=True)
    )
  )
)

In [4]:
start = time.time()
pred = interaction_network(objects, sender_relations, receiver_relations, relation_info)
end = time.time()
pytorch_pred = torch.cat(pred, dim=0)
target = torch.cat(y, dim=0)
pytorch_pred.shape, target.shape, end - start

(torch.Size([75132, 1]), torch.Size([75132, 1]), 1.4913451671600342)

In [5]:
hf = h5py.File('{}/test_{}_{}.hdf5'.format(data_path, prep, pt_cut), 'w')
obj_ms = []
sr_ms = []
rr_ms = []
ri_ms = []
obj_ns = []
sr_ns = []
rr_ns = []
ri_ns = []

for i in range(len(graphs)):
    obj = objects[i].numpy()#.tolist()
    sr = sender_relations[i].numpy()#.tolist()
    rr = receiver_relations[i].numpy()#.tolist()
    ri = relation_info[i].numpy()#.tolist()
    hf.create_dataset('obj_{}'.format(i), data=obj.tolist())
    hf.create_dataset('sr_{}'.format(i), data=sr.tolist())
    hf.create_dataset('rr_{}'.format(i), data=rr.tolist())
    hf.create_dataset('ri_{}'.format(i), data=ri.tolist())
    
    obj_ms.append(obj.shape[0])
    sr_ms.append(sr.shape[0])
    rr_ms.append(rr.shape[0])
    ri_ms.append(ri.shape[0])
    
    obj_ns.append(obj.shape[1])
    sr_ns.append(sr.shape[1])
    rr_ns.append(rr.shape[1])
    ri_ns.append(ri.shape[1])
    
hf.create_dataset('obj_shape_0_i', data=obj_ms)
hf.create_dataset('sr_shape_0_i', data=sr_ms)
hf.create_dataset('rr_shape_0_i', data=rr_ms)
hf.create_dataset('ri_shape_0_i', data=ri_ms)

hf.create_dataset('obj_shape_1_i', data=obj_ns)
hf.create_dataset('sr_shape_1_i', data=sr_ns)
hf.create_dataset('rr_shape_1_i', data=rr_ns)
hf.create_dataset('ri_shape_1_i', data=ri_ns)
hf.close()
hf = h5py.File('{}/model_weights_{}_{}.hdf5'.format(data_path, prep, pt_cut), 'w')
for k in weights.keys():
    weight = weights[k].t()
    hf.create_dataset(k, data=weight.numpy().tolist())
hf.close()

In [6]:
print(len(objects), len(sender_relations), len(receiver_relations), len(relation_info))
for i in range(len(graphs)):
    obj = objects[i].numpy()
    sr = sender_relations[i].numpy()
    rr = receiver_relations[i].numpy()
    ri = relation_info[i].numpy()
    
    print(obj.shape, sr.shape, rr.shape, ri.shape)

1000 1000 1000 1000
(34, 3) (34, 38) (34, 38) (1, 38)
(55, 3) (55, 41) (55, 41) (1, 41)
(51, 3) (51, 59) (51, 59) (1, 59)
(64, 3) (64, 56) (64, 56) (1, 56)
(37, 3) (37, 26) (37, 26) (1, 26)
(82, 3) (82, 75) (82, 75) (1, 75)
(98, 3) (98, 174) (98, 174) (1, 174)
(130, 3) (130, 202) (130, 202) (1, 202)
(76, 3) (76, 104) (76, 104) (1, 104)
(74, 3) (74, 70) (74, 70) (1, 70)
(69, 3) (69, 64) (69, 64) (1, 64)
(40, 3) (40, 31) (40, 31) (1, 31)
(77, 3) (77, 75) (77, 75) (1, 75)
(56, 3) (56, 67) (56, 67) (1, 67)
(62, 3) (62, 84) (62, 84) (1, 84)
(116, 3) (116, 146) (116, 146) (1, 146)
(68, 3) (68, 54) (68, 54) (1, 54)
(62, 3) (62, 47) (62, 47) (1, 47)
(99, 3) (99, 120) (99, 120) (1, 120)
(38, 3) (38, 26) (38, 26) (1, 26)
(81, 3) (81, 68) (81, 68) (1, 68)
(95, 3) (95, 122) (95, 122) (1, 122)
(53, 3) (53, 34) (53, 34) (1, 34)
(40, 3) (40, 37) (40, 37) (1, 37)
(62, 3) (62, 45) (62, 45) (1, 45)
(32, 3) (32, 21) (32, 21) (1, 21)
(69, 3) (69, 65) (69, 65) (1, 65)
(106, 3) (106, 175) (106, 175) (1, 175

(75, 3) (75, 69) (75, 69) (1, 69)
(81, 3) (81, 121) (81, 121) (1, 121)
(30, 3) (30, 22) (30, 22) (1, 22)
(42, 3) (42, 29) (42, 29) (1, 29)
(96, 3) (96, 100) (96, 100) (1, 100)
(61, 3) (61, 46) (61, 46) (1, 46)
(51, 3) (51, 65) (51, 65) (1, 65)
(93, 3) (93, 83) (93, 83) (1, 83)
(30, 3) (30, 21) (30, 21) (1, 21)
(59, 3) (59, 81) (59, 81) (1, 81)
(79, 3) (79, 73) (79, 73) (1, 73)
(35, 3) (35, 24) (35, 24) (1, 24)
(83, 3) (83, 116) (83, 116) (1, 116)
(52, 3) (52, 38) (52, 38) (1, 38)
(55, 3) (55, 37) (55, 37) (1, 37)
(57, 3) (57, 44) (57, 44) (1, 44)
(57, 3) (57, 35) (57, 35) (1, 35)
(41, 3) (41, 29) (41, 29) (1, 29)
(81, 3) (81, 99) (81, 99) (1, 99)
(44, 3) (44, 49) (44, 49) (1, 49)
(32, 3) (32, 39) (32, 39) (1, 39)
(68, 3) (68, 72) (68, 72) (1, 72)
(66, 3) (66, 54) (66, 54) (1, 54)
(94, 3) (94, 142) (94, 142) (1, 142)
(92, 3) (92, 97) (92, 97) (1, 97)
(37, 3) (37, 43) (37, 43) (1, 43)
(78, 3) (78, 65) (78, 65) (1, 65)
(60, 3) (60, 79) (60, 79) (1, 79)
(53, 3) (53, 41) (53, 41) (1, 41)
(5